In [2]:
# ShoeFly.com is performing an A/B Test. They have two different versions of an ad, which they have placed in 
# emails, as well as in banner ads on Facebook, Twitter, and Google. 
# They want to know how the two ads are performing on each of the different platforms on each day of the week.

In [3]:
import pandas as pd

In [7]:
# load the dataframe CSV
ad_clicks = pd.read_csv('Micro project in Python- AB Testing with Pandads_Dataset.csv')
print(ad_clicks.head(3))

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  


In [9]:
# How many views came from each platfrom?
visit_platfrom = ad_clicks.groupby('utm_source').user_id.count().reset_index()
# Rename the columns to be more representative
visit_platfrom.columns = ['platfrom', 'nr_of_views']
print(visit_platfrom)

   platfrom  nr_of_views
0     email          255
1  facebook          504
2    google          680
3   twitter          215


In [12]:
# Which platfrom drived the most visits?
max_view = visit_platfrom[visit_platfrom.nr_of_views == visit_platfrom.nr_of_views.max()]
print(max_view)

  platfrom  nr_of_views
2   google          680


In [13]:
#Real visits
ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()
print(ad_clicks.head(3))

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   

  ad_click_timestamp experimental_group  is_click  
0               7:18                  A      True  
1                NaN                  B     False  
2                NaN                  A     False  


In [15]:
# The percent of real people who clicked on ads from each platform
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()
# print(clicks_by_source)
# Pivot the data
clicks_pivot = clicks_by_source.pivot(columns = 'is_click', index = 'utm_source', values='user_id').reset_index()
print(clicks_pivot)
clicks_pivot['percent_clicked'] = clicks_pivot[True] / (clicks_pivot[True] + clicks_pivot[False])
print(clicks_pivot)

is_click utm_source  False  True
0             email    175    80
1          facebook    324   180
2            google    441   239
3           twitter    149    66
is_click utm_source  False  True  percent_clicked
0             email    175    80         0.313725
1          facebook    324   180         0.357143
2            google    441   239         0.351471
3           twitter    149    66         0.306977


In [16]:
# A versus B ads, display and click 
A_B = ad_clicks.groupby('experimental_group').user_id.count()
print(A_B)
A_B_actual = ad_clicks.groupby(['experimental_group','is_click']).user_id.count()
print(A_B_actual)

experimental_group
A    827
B    827
Name: user_id, dtype: int64
experimental_group  is_click
A                   False       517
                    True        310
B                   False       572
                    True        255
Name: user_id, dtype: int64


In [20]:
# clicks per day for each ad type A and B
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']
print(a_clicks.head(2))
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']
print(b_clicks.head(2))

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   

  ad_click_timestamp experimental_group  is_click  
0               7:18                  A      True  
2                NaN                  A     False  
                                user_id utm_source          day  \
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook   7 - Sunday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google  2 - Tuesday   

  ad_click_timestamp experimental_group  is_click  
1                NaN                  B     False  
3                NaN                  B     False  
